<a href="https://colab.research.google.com/github/HaswanthVIT/BakeScape/blob/main/BakeScape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install diffusers transformers accelerate xformers
!pip install pandas opencv-python Pillow scikit-image matplotlib
!pip install gradio wandb imagehash


Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 9.0 MB/s eta 0:00:00


In [2]:
# Create clean folder structure
!mkdir -p bakescape/{src,configs}
!mkdir -p data/{raw,processed}
!mkdir -p outputs/{gen,logs,ckpts}
!touch bakescape/__init__.py
